# Data Preprocessing

# 1. Import All Necessary Libraries and Create File Paths

In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from osgeo import gdal, gdal_array
import sys
import cv2
import tqdm

In [42]:
# folder path or file path constants that will be used in this project
# Root Directory (between 'F:' or 'D:'' depends on the external drive)
ROOT = "D:\\"

# Folder inside D:\\wildfire-sumatera-dataset
WILDFIRE_SUMATERA_DATASET_FOLDER_PATH = os.path.join(ROOT, 'wildfire-sumatera-dataset')


# Folders and metadatas inside D:\\wildfire-sumatera-dataset
WILDFIRE_SUMATERA_GEOTIFF_FOLDER_PATH = os.path.join(WILDFIRE_SUMATERA_DATASET_FOLDER_PATH, 'wildfire-sumatera-geotiff')
WILDFIRE_SUMATERA_IMAGE_FOLDER_PATH = os.path.join(WILDFIRE_SUMATERA_DATASET_FOLDER_PATH, 'wildfire-sumatera-image')
WILDFIRE_SUMATERA_IMAGE_MASK_FOLDER_PATH = os.path.join(WILDFIRE_SUMATERA_DATASET_FOLDER_PATH, 'wildfire-sumatera-image-mask')
WILDFIRE_SUMATERA_IMAGE_MASK_TFRECORD_FOLDER_PATH = os.path.join(WILDFIRE_SUMATERA_DATASET_FOLDER_PATH, 'wildfire-sumatera-image-mask-tfrecords')

# Files (.csv) and metadatas inside D:\\wildfire-sumatera-dataset
METADATA_LANDSAT_8_FILE_PATH = os.path.join(WILDFIRE_SUMATERA_DATASET_FOLDER_PATH, 'metadata_landsat_8.csv')
METADATA_SENTINEL_2_FILE_PATH = os.path.join(WILDFIRE_SUMATERA_DATASET_FOLDER_PATH, 'metadata_sentinel_2.csv')

# Folders inside D:\\wildfire-sumatera-dataset\\wildfire-sumatera-geotiff
SENTINEL_2_GEOTIFF_FOLDER_PATH = os.path.join(WILDFIRE_SUMATERA_GEOTIFF_FOLDER_PATH, 'sentinel-2')
LANDSAT_8_GEOTIFF_FOLDER_PATH = os.path.join(WILDFIRE_SUMATERA_GEOTIFF_FOLDER_PATH, 'landsat-8')

# Folders inside D:\\wildfire-sumatera-dataset\\wildfire-sumatera-image
SENTINEL_2_IMAGE_FOLDER_PATH = os.path.join(WILDFIRE_SUMATERA_IMAGE_FOLDER_PATH, 'sentinel-2')
LANDSAT_8_IMAGE_FOLDER_PATH = os.path.join(WILDFIRE_SUMATERA_IMAGE_FOLDER_PATH, 'landsat-8')

# Folders inside D:\\wildfire-sumatera-dataset\\wildfire-sumatera-geotiff\\landsat-8
LANDSAT_8_PREFIRE_GEOTIFF_FOLDER_PATH = os.path.join(LANDSAT_8_GEOTIFF_FOLDER_PATH, 'prefire')
LANDSAT_8_POSTFIRE_GEOTIFF_FOLDER_PATH = os.path.join(LANDSAT_8_GEOTIFF_FOLDER_PATH, 'postfire')

# Folders inside D:\\wildfire-sumatera-dataset\\wildfire-sumatera-geotiff\\sentinel-2
SENTINEL_2_PREFIRE_GEOTIFF_FOLDER_PATH = os.path.join(SENTINEL_2_GEOTIFF_FOLDER_PATH, 'prefire')
SENTINEL_2_POSTFIRE_GEOTIFF_FOLDER_PATH = os.path.join(SENTINEL_2_GEOTIFF_FOLDER_PATH, 'postfire')

# Folders inside D:\\wildfire-sumatera-dataset\\wildfire-sumatera-image-mask
SENTINEL_2_IMAGE_MASK_FOLDER_PATH = os.path.join(WILDFIRE_SUMATERA_IMAGE_MASK_FOLDER_PATH, 'sentinel-2')
LANDSAT_8_IMAGE_MASK_FOLDER_PATH = os.path.join(WILDFIRE_SUMATERA_IMAGE_MASK_FOLDER_PATH, 'landsat-8')

# Folders inside
SENTINEL_2_IMAGE_MASK_TFRECORD_FOLDER_PATH = os.path.join(WILDFIRE_SUMATERA_IMAGE_MASK_TFRECORD_FOLDER_PATH, 'sentinel-2')
LANDSAT_8_IMAGE_MASK_TFRECORD_FOLDER_PATH = os.path.join(WILDFIRE_SUMATERA_IMAGE_MASK_TFRECORD_FOLDER_PATH, 'landsat-8')

dirs = [
    WILDFIRE_SUMATERA_DATASET_FOLDER_PATH,
    WILDFIRE_SUMATERA_GEOTIFF_FOLDER_PATH, 
    WILDFIRE_SUMATERA_IMAGE_FOLDER_PATH,
    WILDFIRE_SUMATERA_IMAGE_MASK_FOLDER_PATH,
    WILDFIRE_SUMATERA_IMAGE_MASK_TFRECORD_FOLDER_PATH,
    
    SENTINEL_2_GEOTIFF_FOLDER_PATH, 
    LANDSAT_8_GEOTIFF_FOLDER_PATH,
    SENTINEL_2_IMAGE_FOLDER_PATH,
    LANDSAT_8_IMAGE_FOLDER_PATH,
    
    LANDSAT_8_PREFIRE_GEOTIFF_FOLDER_PATH,
    LANDSAT_8_POSTFIRE_GEOTIFF_FOLDER_PATH,
    SENTINEL_2_PREFIRE_GEOTIFF_FOLDER_PATH,
    SENTINEL_2_POSTFIRE_GEOTIFF_FOLDER_PATH,
    
    SENTINEL_2_IMAGE_MASK_FOLDER_PATH,
    LANDSAT_8_IMAGE_MASK_FOLDER_PATH,
    
    SENTINEL_2_IMAGE_MASK_TFRECORD_FOLDER_PATH,
    LANDSAT_8_IMAGE_MASK_TFRECORD_FOLDER_PATH,
]

for dir_ in dirs:
    if not os.path.exists(dir_):
        os.mkdir(dir_)
        print(f"{dir_} has been created")
    else:
        print(f"{dir_} already exist")

# 2. Access The Dataset
The dataset name is **data-total.csv** which contains all the necessary information about dates and location coordinates of a wildfire events in Indonesia.

In [40]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

wf_data  = pd.read_csv("../data-total.csv")
wf_data.columns

# rename columns
wf_data.rename(
    columns={
        'Lat': 'lat', 
        'Long': 'long', 
        'Satellite': 'satellite', 
        'Time_UTC': 'time_utc', 
        'Date': 'date', 
        'Source': 'source', 
        'PROVINSI': 'provinsi',
        'KAB_KOTA': 'kab_kota'
        }, 
    inplace=True,
    )

# adding '-' to date column values
wf_data.date = pd.to_datetime(wf_data.date, format="%Y%m%d")
wf_data.date = wf_data.date.astype('string')

# All provinces in sumatera
sumatera_provinces = [
    'SUMATERA UTARA',
    'JAMBI',
    'RIAU',
    'SUMATERA BARAT',
    'SUMATERA SELATAN',
    'BENGKULU',
    'LAMPUNG',
    'ACEH',
    ]

# create filter so that only events in sumatera will be returned.
sumatera_filter = wf_data['provinsi'].apply(lambda x: x in sumatera_provinces) 
sumatera_only = wf_data.loc[sumatera_filter]

display(sumatera_only.info())
display(sumatera_only.provinsi.unique())
display(sumatera_only.head())

In [41]:
# sort the data in based on 'date' column in ascending order
sumatera_only = sumatera_only.sort_values(
    by=['date'], 
    ascending=True,
    ignore_index=True,
)

sumatera_only.head()

# 3. Create All Functions for the Project and Constants

In [6]:
def get_img_as_array(filepath, dictionary):
    """
    :param filepath: location of the satellite bands
    :param dictionary: information about the image bands
    :return: dictionary containing name of the satellite bands as key and array of pixels as value  
    """
    os.chdir(filepath)
    print(filepath)
    band_imgs = {}
    for key, val in dictionary.items():
        # open the .tiff file
        if os.path.exists(filepath):
            tiff_file = gdal.Open(key)
            bands = key.split('_')
            for i, band in enumerate(bands):
                band_imgs[band] = tiff_file.GetRasterBand(i+1).ReadAsArray()
    
    return band_imgs

In [7]:
def save_satellite_image(bands=[], file_path=""):
    """
    :param bands: array of satellite image bands
    :param filepath: location of the image being stored
    :return: return result of the conversion (True which is 'success' or False which is 'failed') 
    """
    if not bands:
        print("bands cannot be empty")
        return 
    
    if not file_path:
        print("filename cannot be empty")
        return
    img = cv2.merge(bands)
    img *= 255
    cv2.imwrite(file_path, img)
    return True

In [8]:
LANDSAT_POSTFIRE_DICT = {
    'dnbr_b7': 2,
    'b5_b4': 2,
    'b3_b2': 2
}

SENTINEL_POSTFIRE_DICT = {
    'dnbr_b12': 2,
    'b8_b4': 2,
    'b3_b2': 2
}

# 4. Open Landsat CSV File

In [39]:
PNG_EXT = '.png'
landsat_df = pd.read_csv(METADATA_LANDSAT_8_FILE_PATH)
landsat_df.head()

## 4.1 Check Image Condition From All Bands
Filter to check whether all of the array forms of each bands is not **None** or **empty**

In [10]:
landsat_image_condition = []
for index, row in landsat_df.iterrows():
    # open the image path
    condition = 1
    landsat_postfire_img_band = get_img_as_array(
        os.path.join(ROOT, row['folder_paths_geotiff']), 
        LANDSAT_POSTFIRE_DICT,
    )
    defect = [val is None for key, val in landsat_postfire_img_band.items()]            
    if np.sum(defect) > 0:
        condition=0
        print("defect detected")
    landsat_image_condition.append(condition)

In [ ]:
landsat_df['image_condition'] = landsat_image_condition
landsat_df.to_csv(METADATA_LANDSAT_8_FILE_PATH, index=False)

## 4.2 Binarized Image
Save a file containing real images with their masks

In [38]:
for index, row in landsat_df.iterrows():
    print(row['folder_paths_geotiff'])
    if row['image_condition']:
        # create directory path
        directory = os.path.join(LANDSAT_8_IMAGE_MASK_FOLDER_PATH, str(index))
        print(directory)

        # check if the directory already exists
        if not os.path.exists(directory):
            os.mkdir(directory)
            print("{} has been created".format(directory))
        else:
            print("{} already exist".format(directory))
        
        # check if image_b4_b3_b2.png, image_b7_b5_b2.png, and mask.png already exists
        image_b4_b3_b2_path = os.path.join(directory, 'image_b4_b3_b2.png')
        image_b7_b5_b2_path = os.path.join(directory, 'image_b7_b5_b2.png')
        mask_path = os.path.join(directory, 'mask.png')
        if os.path.exists(image_b4_b3_b2_path) and os.path.exists(image_b7_b5_b2_path) and os.path.exists(mask_path):
            print("{}, {}, and {} already exist".format(image_b4_b3_b2_path, image_b7_b5_b2_path, mask_path))
            continue
            
        # open the image path
        landsat_postfire_img_band = get_img_as_array(
            os.path.join(ROOT, row['folder_paths_geotiff']), 
            LANDSAT_POSTFIRE_DICT,
        )
        mask_binary = np.array(landsat_postfire_img_band['dnbr'], dtype=np.float32)
        mask_binary = np.where(mask_binary<0.05, 0, 255)
        
        # save image using band b4, b3, and b2
        save_satellite_image([
            landsat_postfire_img_band['b2'], 
            landsat_postfire_img_band['b3'],
            landsat_postfire_img_band['b4'],], 
            os.path.join(directory, image_b4_b3_b2_path))
        
        # save image using band b7, b5, and b2
        save_satellite_image([
            landsat_postfire_img_band['b2'], 
            landsat_postfire_img_band['b5'],
            landsat_postfire_img_band['b7'],], 
            os.path.join(directory, image_b7_b5_b2_path))
        
        # save mask
        cv2.imwrite(os.path.join(directory, mask_path), mask_binary)
        break
    else:
        print('image is broke')

Create a new column that contains the location of images with their masks

In [43]:
landsat_8_image_mask_folders = [os.path.join('\\', 'wildfire-sumatera-dataset', 'wildfire-sumatera-image-mask', 'landsat-8', str(i)) for i in range(len(landsat_df.index))]
landsat_df['folder_paths_image_mask'] = landsat_8_image_mask_folders
landsat_df.to_csv(METADATA_LANDSAT_8_FILE_PATH, index=False)

## 4.3 Clean Images by Area of Pixels and Save as PNG
small area of pixels will be removed to reduce noise

In [35]:
for index, row in landsat_df.iterrows():
    if row['image_condition']:
        # create directory path
        directory = os.path.join(ROOT, row['folder_paths_image_mask'])
        print(directory)
        
        # check if the image.png and mask.png already exists
        cleaned_mask_path = os.path.join(directory, 'cleaned_mask.png')
        if os.path.exists(cleaned_mask_path):
            print("{} already exist".format(cleaned_mask_path))
            continue
        
        image = cv2.imread(os.path.join(directory, 'mask.png'))

        # convert image to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # apply GaussianBlur to the image
        blur = cv2.GaussianBlur(gray, (7, 7), 0)
        
        # create a binary representation of the image
        # thresholding means all pixels below T (threshold value) will be 0
        # else will be 255
        thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        
        # Filter using contour area and remove small noise
        cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        for c in cnts:
            area = cv2.contourArea(c)
            if area < 5500:
                cv2.drawContours(thresh, [c], -1, (0,0,0), -1)

        # Morph close and invert image
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
        cleaned_mask = 255 - cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
        cv2.imwrite(os.path.join(directory, cleaned_mask_path), cleaned_mask)
        break

# 5. Open Sentinel CSV File

In [28]:
PNG_EXT = '.png'
sentinel_df = pd.read_csv(METADATA_SENTINEL_2_FILE_PATH)
sentinel_df.head()

,folder_paths_geotiff,mean_dnbr,median_dnbr,class_by_mean,class_by_median,image_condition,folder_paths_image_mask
0,\wildfire-sumatera-dataset\wildfire-sumatera-g...,0.124348,0.098779,0,0,1,\wildfire-sumatera-dataset\wildfire-sumatera-i...
1,\wildfire-sumatera-dataset\wildfire-sumatera-g...,0.124229,0.098487,0,0,1,\wildfire-sumatera-dataset\wildfire-sumatera-i...
2,\wildfire-sumatera-dataset\wildfire-sumatera-g...,0.123137,0.096676,0,0,1,\wildfire-sumatera-dataset\wildfire-sumatera-i...
3,\wildfire-sumatera-dataset\wildfire-sumatera-g...,0.141992,0.104607,0,0,1,\wildfire-sumatera-dataset\wildfire-sumatera-i...
4,\wildfire-sumatera-dataset\wildfire-sumatera-g...,0.124485,0.098878,0,0,1,\wildfire-sumatera-dataset\wildfire-sumatera-i...


## 5.1 Check Image Condition From All Bands
Filter to check whether all of the array forms of each bands is not **None** or **empty**

In [18]:
sentinel_image_condition = []
for index, row in sentinel_df.iterrows():
    # open the image path
    condition = 1
    sentinel_postfire_img_band = get_img_as_array(
        os.path.join(ROOT, row['folder_paths_geotiff']), 
        SENTINEL_POSTFIRE_DICT,
    )
    defect = [val is None for key, val in sentinel_postfire_img_band.items()]            
    if np.sum(defect) > 0:
        condition=0
        print("defect detected")
    sentinel_image_condition.append(condition)

In [12]:
sentinel_df['image_condition'] = sentinel_image_condition
sentinel_df.to_csv(METADATA_SENTINEL_2_FILE_PATH, index=False)

## 5.2 Binarized Image
Save a file containing real images with their masks

In [36]:
for index, row in sentinel_df.iterrows():
    print(row['folder_paths_geotiff'])
    if row['image_condition']:
        # create directory path
        directory = os.path.join(SENTINEL_2_IMAGE_MASK_FOLDER_PATH, str(index))

        # check if the directory already exists
        if not os.path.exists(directory):
            os.mkdir(directory)
            print("{} has been created".format(directory))
        else:
            print("{} already exist".format(directory))

        # check if image_b4_b3_b2.png, image_b12_b8_b2.png, and mask.png already exists        
        image_b4_b3_b2_path = os.path.join(directory, 'image_b4_b3_b2.png')
        image_b12_b8_b2_path = os.path.join(directory, 'image_b12_b8_b2.png')
        mask_path = os.path.join(directory, 'mask.png')
        if os.path.exists(image_b4_b3_b2_path) and os.path.exists(image_b12_b8_b2_path) and os.path.exists(mask_path):
            print("{}, {}, and {} already exist".format(image_b4_b3_b2_path, image_b12_b8_b2_path, mask_path))            
            continue

        # open the image path
        sentinel_postfire_img_band = get_img_as_array(
            os.path.join(ROOT, row['folder_paths_geotiff']), 
            SENTINEL_POSTFIRE_DICT,
        )
        mask_binary = np.array(sentinel_postfire_img_band['dnbr'], dtype=np.float32)
        mask_binary = np.where(mask_binary<0.05, 0, 255)        
        # save image using band b4, b3, and b2
        save_satellite_image([
            sentinel_postfire_img_band['b2'], 
            sentinel_postfire_img_band['b3'],
            sentinel_postfire_img_band['b4'],], 
            os.path.join(directory, image_b4_b3_b2_path))
        
        # save image using band b12, b8, and b2
        save_satellite_image([
            sentinel_postfire_img_band['b2'], 
            sentinel_postfire_img_band['b8'],
            sentinel_postfire_img_band['b12'],], 
            os.path.join(directory, image_b12_b8_b2_path))
        
        # save mask
        cv2.imwrite(os.path.join(directory, 'mask{}'.format(PNG_EXT)), mask_binary)
        break
    else:
        print('image is broke')

In [15]:
sentinel_image_mask_folders = [os.path.join('\\', 'wildfire-sumatera-dataset', 'wildfire-sumatera-image-mask', 'sentinel-2', str(i)) for i in range(len(sentinel_df.index))]
sentinel_df['folder_paths_image_mask'] = sentinel_image_mask_folders
sentinel_df.to_csv(METADATA_SENTINEL_2_FILE_PATH, index=False)

## 5.3 Clean Images by Area of Pixels and Save as PNG
small area of pixels will be removed to reduce noise

In [37]:
for index, row in sentinel_df.iterrows():
    if row['image_condition']:
        # create directory path
        directory = os.path.join(ROOT, row['folder_paths_image_mask'])
        print(directory)

        
        # check if the image.png and mask.png already exists
        cleaned_mask_path = os.path.join(directory, 'cleaned_mask.png')
        if os.path.exists(cleaned_mask_path):
            print("{} already exist".format(cleaned_mask_path))
            continue
        
        image = cv2.imread(os.path.join(directory, 'mask.png'))

        # convert image to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # apply GaussianBlur to the image
        blur = cv2.GaussianBlur(gray, (7, 7), 0)
        
        # create a binary representation of the image
        # thresholding means all pixels below T (threshold value) will be 0
        # else will be 255
        thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        # Filter using contour area and remove small noise
        cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        for c in cnts:
            area = cv2.contourArea(c)
            if area < 5500:
                cv2.drawContours(thresh, [c], -1, (0,0,0), -1)

        # Morph close and invert image
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
        cleaned_mask = 255 - cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
        cv2.imwrite(os.path.join(directory, cleaned_mask_path), cleaned_mask)